In [3]:
# Ariel Hasse, David Hitlin, SURF; 
# UV full spectrum extended; 1460 Volts
# This program pulls the channel number and energy count for each source in the specified PMT
# set-up. It fits the energy peaks to a gaussian and records the peak value to find the calibration
# between known source energy and channel number. This slope and the expected radium energy peaks
# create the quenching factor for each energy level. The final output is a txt file with the resolutions
# of the peaks, the gaussian parameters, the calibration factor, and the quenching factors and all png 
# files for each plot produced.


%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.integrate as integrate
from scipy.optimize import minimize 
from scipy import stats
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy import interp

In [4]:
def retrieve_data(file):
    '''This function takes data from a file and outputs the columns into two lists, x and y. 
    The file must have two columns of data seperated by a tab. The argument requires a 
    docstring for the file name.'''
    
    # A place hold list, a place hold string, the first column value in string, the second 
    # column value in string, x values list, and y values list
    point = []
    num = ''
    first = ''
    second = ''
    x = []
    y = []
    
    # This opens the file and reads the first line to discard the first value
    f = open(file, 'r')
    f.readline()
    
    # This will loop through each line to extract the x and y values and append it to the 
    # x and y lists
    for line in f:
        num = ''
        point = list(line)
        
        # This loop removes the first value in point and the tab in between the columns.
        for i in range(len(point)):
            if point[0] == '\t':
                del point[0]
                first = num
                break
            else: 
                num += point[0]
                del point[0]
        num = ''
        
        # This loop saves each column in the list as a string and creates an integer
        # of the second value. It saves the placeholder num value to second when the tab
        # at the end of the file occurs. 
        for i in range(len(point)):
            if point[0] == '\t':
                del point[0]
                second = num
                break
            else: 
                num += point[0]
                del point[0]
        x.append(float(first))
        y.append(float(second))
        
    return x, y

In [5]:
def data_interval(x, y, upper, lower):
    ''' This function takes two lists, x an y, and two integers, upper and lower.
    It outputs two lists within the upper and lower bounds (determined by the x variable)''' 
    
    # counter for y index, list for segmented x values, list for segmented y values
    count = 0
    newx = []
    newy = []
    for i in x:
        if i > lower:
                if i < upper:
                    newx.append(i)
                    newy.append(y[count])
        count += 1
    return newx, newy

In [6]:
def plot_data(x, y, title, xname, yname, legend):
    ''' This function takes two lists and graphing information and plots the 
    lists against eachother'''
    
    plt.plot(x, y, label = legend)
    plt.title(title)
    plt.xlabel(xname)
    plt.ylabel(yname)
    plt.legend()
    plt.show()

In [25]:
def gauss1(guess):
    '''This is a gaussian chi squared sum. The argument requires a list of
    three variables and x and y lists must be defined outside of the function
    in the porgram. Its a single gaussian and can be passed to optimize.'''
    
    # This function is passed into optimize which uses the minimize function which
    # only takes arrays.
    arx = np.asarray(x1)
    ary = np.asarray(y1)
    
    return sum(np.power(guess[0]*np.exp(-np.power((arx-guess[1]), 2)/(2*np.power(guess[2], 2))) + guess[3]*arx + guess[4] - ary, 2))
    

In [10]:
def optimize(f, guess):
    '''Takes a function and potential parameters and outputs the minimized 
    values for the guess'''
    fit = minimize(f, guess, method= 'Nelder-Mead', bounds = ((0, None), (0, None), (0, None), (None, 0), (0, None)))
    return fit
   
  

In [12]:
def linefitgauss(opt, x, title, xname, yname, legend):
    ''' This plots the line of best fit from the optimize function for a 
    ploynomial'''
    ans = list(opt.x)
    i = 0
    y = []
    for i in x:
        y.append(ans[0]*np.exp(-np.power((i-ans[1]), 2)/(2*np.power(ans[2], 2)))+ ans[3]*i + ans[4])
    plt.plot(x, y)
    plt.plot(x, y, label = legend)
    plt.title(title)
    plt.xlabel(xname)
    plt.ylabel(yname)
    plt.legend()
    plt.show()

 

In [13]:
def linefitgaussRn(opt, x, title, xname, yname, legend):
    ''' This plots the line of best fit from the optimize function for a 
    ploynomial'''
    ans = list(opt.x)
    i = 0
    y = []
    for i in x:
        y.append(ans[0]*np.exp(-np.power((i-ans[1]), 2)/(2*np.power(ans[2], 2)))+\
            ans[3]*np.exp(-np.power((i-ans[4]), 2)/(2*np.power(ans[5], 2))) +\
            ans[6]*np.exp(-np.power((i-ans[7]), 2)/(2*np.power(ans[8], 2)))+ ans[9]*i + ans[10])
    plt.plot(x, y)
    plt.plot(x, y, label = legend)
    plt.title(title)
    plt.xlabel(xname)
    plt.ylabel(yname)
    plt.legend()
    plt.show()        
 

In [14]:
def gaussRn(guess):
    '''This is a gaussian chi squared sum. The argument requires a list of
    three variables and x and y lists must be defined outside of the function
    in the porgram'''
    arx = np.asarray(x1)
    ary = np.asarray(y1)
    return sum(np.power(guess[0]*np.exp(-np.power((arx-guess[1]), 2)/(2*np.power(guess[2], 2)))\
        + guess[3]*np.exp(-np.power((arx-guess[4]), 2)/(2*np.power(guess[5], 2))) + \
        guess[6]*np.exp(-np.power((arx-guess[7]), 2)/(2*np.power(guess[8], 2))) + arx*guess[9] + guess[10] - ary, 2))
                                                      
                                           

In [15]:
def resolution(opt):
    '''This finds the resolution of the fit using the width, standard deviation
    and the center, the mean'''
    values = list(opt.x)
    return values[2]/values[1]
    

In [16]:
def conversion(opt, y):
    '''Automated Line of Best Fit for gamma and electron sources'''
    y = []
    x = []
    rawx =[]
    for i in opt:
        rawx.append(i)
    for i in rawx:
        x.append(i[1])
    for i in y:
        y.append(i)
    plt.errorbar(x, y, xerr = (opt[0][0], opt[1][0], opt[2][0]))
    plt.scipy.stats.linregress(x, y)
    plt.show()

In [17]:
def line(x, i, j):
    ''' A linear function'''
    return i*x + j  

In [18]:
def chan_kev():
    '''By replacing the y, and err values the function plots the points with
    their error and plots the line of bestfit. The error values are not correct'''
    x = [60, 511, 662, 1170, 1274, 1330]
    y = [304, 2343.6, 3445, 5772.8, 6373.4, 6648.8]
    err = [59.6, 138, 163.7, 560.1, 224.8, 239.8]
    names = [' ', 'Na-22 First  ', 'Cs-137  ', 'Co-60 First  ', 'Na-22 Second  ', 'Co-60 Second  ']
    first = ['  AmBe-241']
    yfit = []
    linfit = curve_fit(line, x, y, guessregress, sigma = err)
    for i in range(1400):
        yfit.append(i*linfit[0][0] + linfit[0][1])
    plt.errorbar(x, y, yerr= err, linestyle = 'None', marker = 'o', label = 'Raw Data Points')
    plt.plot(range(1400), yfit, label = 'Line of Best Fit')
    plt.ylim(-100, 17000)
    plt.title('Channel to Kev Count for Four Radioactive Sources')
    plt.xlabel('Known Particle Energy (KeV)')
    plt.ylabel('Channels (ADC)')
    plt.legend(loc = 2)
    #for i, txt in enumerate(names):
        #plt.annotate(txt, (x[i],y[i]), ha = 'right')
        #plt.show()
    #for i, txt in enumerate(first):
        #plt.annotate(txt, (x[i], y[i]), ha = 'left')
        #plt.show()
    plt.show()    
    print linfit
  

In [19]:
def radium():
    '''x represents the known keV values for the Ra impurities and y represents 
    the measured values in channels. This function outputs the quenching factor 
    for each measurement'''
    known = [4800, 5500, 6000, 7700]
    measured = [7317.1, 8740.1, 10044.8, 14202]
    error1 = [273.5, 389.5, 300.2, 356.8]
    error2 = []
    names = ['Ra-226    ', 'Ra-222    ', 'Po-218   ', 'Po-214    ']
    coun = 0
    tick = 0
    quench = []
    yfit = []
    for i in measured:
        pred = i*0.200634
        quench.append(known[coun]/pred)
        coun += 1
    linfit = stats.linregress(known, quench)
    for i in range(8000):
        yfit.append(i*linfit[0] + linfit[1])
    for i in error1:
        exper = measured[tick]
        recor = known[tick]
        error2.append((i*recor)/(math.pow(exper, 2)))
        tick += 1
    plt.plot(range(8000), yfit, label = 'Line of Best Fit')
    plt.xlim(4000, 8000)
    plt.errorbar(known, quench, yerr = error2, marker = 'o', linestyle = 'None', label = 'Quenching Factor')
    plt.title('Quenching Factor at Various Energies for Ra 226 in BaF2')
    plt.xlabel('Energy in keV')
    plt.ylabel('Quenching Factor (Birks Constant)')
    plt.legend(loc = 1)
    #for i, txt in enumerate(names):
        #plt.annotate(txt, (known[i],quench[i]), ha = 'right')
        #plt.show()
    plt.show()
    return quench
    
    

 

In [20]:
# AmBe 241

poly = [-.000001, .2, -1600]

hyp = [5000, 300, 200, -.00001, -.0001]

threepeaks = [40, 14000, 1000, 40, 17000, 1000, 40, 20000, 1000, 0, 0]

x1 = energy(alldata('AmBe2UV1460.txt')[0], alldata('AmBe2UV1460.txt')[1], 500, 150)[0]
y1 = energy(alldata('AmBe2UV1460.txt')[0], alldata('AmBe2UV1460.txt')[1], 500, 150)[1]  
        
rawdata(x1, y1, 'AmBe 60 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Collected Data (Observed)')
fitted = optimize(gauss, hyp)
linefitgauss(fitted, x1, 'AmBe 60 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Gaussian Fit to Data')
print resolution(fitted)
print fitted.x

NameError: name 'energy' is not defined

In [ ]:
# Cs 137

poly = [-.000001, .2, -1600]

hyp = [500, 3400, 800, -.00001, -.0001]

threepeaks = [40, 14000, 1000, 40, 17000, 1000, 40, 20000, 1000, 0, 0]

x1 = energy(alldata('Cs2UV1460.txt')[0], alldata('Cs2UV1460.txt')[1], 4000, 3000)[0]
y1 = energy(alldata('Cs2UV1460.txt')[0], alldata('Cs2UV1460.txt')[1], 4000, 3000)[1]  
        
rawdata(x1, y1, 'Cs 137 662 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Collected Data (Observed)')
fitted = optimize(gauss, hyp)
linefitgauss(fitted, x1, 'Cs 137 662 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Gaussian Fit to Data')
print resolution(fitted)
print fitted.x

In [ ]:
# Co 60 First Peak

poly = [-.000001, .2, -1600]

hyp = [3500, 5900, 300, -.45, -.0001]

threepeaks = [40, 14000, 1000, 40, 17000, 1000, 40, 20000, 1000, 0, 0]

x1 = energy(alldata('Co2UV1460.txt')[0], alldata('Co2UV1460.txt')[1], 6200, 5500)[0]
y1 = energy(alldata('Co2UV1460.txt')[0], alldata('Co2UV1460.txt')[1], 6200, 5500)[1]  
        

rawdata(x1, y1, 'Co 60 1170 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Collected Data (Observed)')
fitted = optimize(gauss, hyp)
linefitgauss(fitted, x1, 'Co 60 1170 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Gaussian Fit to Data')
print resolution(fitted)
print fitted.x

In [ ]:
# Co 60 Second Peak

poly = [-.000001, .2, -1600]

hyp = [1500, 6700, 800, -.09, -.0001]

threepeaks = [40, 14000, 1000, 40, 17000, 1000, 40, 20000, 1000, 0, 0]

x1 = energy(alldata('Co2UV1460.txt')[0], alldata('Co2UV1460.txt')[1], 7200, 6300)[0]
y1 = energy(alldata('Co2UV1460.txt')[0], alldata('Co2UV1460.txt')[1], 7200, 6300)[1]  
        

rawdata(x1, y1, 'Co 60 1330 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Collected Data (Observed)')
fitted = optimize(gauss, hyp)
linefitgauss(fitted, x1, 'Co 60 1330 keV Peak at 1350 Volts', 'Energy in Channels', 'Counts', 'Gaussian Fit to Data')
print resolution(fitted)
print fitted.x

In [ ]:
# Na 22 First Peak

poly = [-.000001, .2, -1600]

hyp = [2500, 2600, 600, -.00001, -.0001]

threepeaks = [40, 14000, 1000, 40, 17000, 1000, 40, 20000, 1000, 0, 0]

x1 = energy(alldata('Na2UV1460.txt')[0], alldata('Na2UV1460.txt')[1], 3000, 2100)[0]
y1 = energy(alldata('Na2UV1460.txt')[0], alldata('Na2UV1460.txt')[1], 3000, 2100)[1]  
        
rawdata(x1, y1, 'Na 22 511 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Collected Data (Observed)')
fitted = optimize(gauss, hyp)
linefitgauss(fitted, x1, 'Na 22 511 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Gaussian Fit to Data')
print resolution(fitted)
print fitted.x

In [ ]:
# Na 22 Second Peak

poly = [-.000001, .2, -1600]

hyp = [150, 6400, 800, -.0001, -.0001]

threepeaks = [40, 14000, 1000, 40, 17000, 1000, 40, 20000, 1000, 0, 0]

x1 = energy(alldata('Na2UV1460.txt')[0], alldata('Na2UV1460.txt')[1], 7000, 5800)[0]
y1 = energy(alldata('Na2UV1460.txt')[0], alldata('Na2UV1460.txt')[1], 7000, 5800)[1]  
        
rawdata(x1, y1, 'Na 22 1274 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Collected Data (Observed)')
fitted = optimize(gauss, hyp)
linefitgauss(fitted, x1, 'Na 22 1274 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Gaussian Fit to Data')
print resolution(fitted)
print fitted.x

In [ ]:
# Ra 226 First Three Peaks

threepeaks = [254, 7300, 1000, 300, 8600, 1200, 260, 10500, 1100, 0, 0]

x1 = energy(alldata('Ra2UV1460.txt')[0], alldata('Ra2UV1460.txt')[1], 11000, 6500)[0]
y1 = energy(alldata('Ra2UV1460.txt')[0], alldata('Ra2UV1460.txt')[1], 11000, 6500)[1]  
        
rawdata(x1, y1, 'Ra 226 4800, 5500, and 6000 keV Peaks at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Collected Data (Observed)')
fitted = optimize(gaussRn, threepeaks)
linefitgaussRn(fitted, x1, 'Ra 226 4800, 5500, and 6000 keV Peaks at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Gaussian Fit to Data')
print resolution(fitted)
print fitted.x

In [ ]:
# Ra 226 Fourth Peak

poly = [-.000001, .2, -1600]

hyp = [500, 14300, 1500, -.000001, -.0001]

threepeaks = [80, 10000, 1200, 80, 12000, 1200, 80, 14200, 1200, 0, 0]

x1 = energy(alldata('Ra2UV1460.txt')[0], alldata('Ra2UV1460.txt')[1], 15500, 13000)[0]
y1 = energy(alldata('Ra2UV1460.txt')[0], alldata('Ra2UV1460.txt')[1], 15500, 13000)[1]  
        
rawdata(x1, y1, 'Ra 7700 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Collected Data (Observed)')
fitted = optimize(gauss, hyp)
linefitgauss(fitted, x1, 'Ra 226 7700 keV Peak at 1460 Volts 2UV', 'Energy in Channels', 'Counts', 'Gaussian Fit to Data')
print resolution(fitted)
print fitted.x

In [ ]:
guessregress = [11, 50]
chan_kev()

In [ ]:
radium()